In [1]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import pyarrow as pa
import csv
import subprocess

#from tutorial on https://community.cloudera.com/t5/Community-Articles/Interacting-with-Hadoop-HDFS-using-Python-codes/ta-p/245163
def run_cmd(args_list):
        """
        run linux commands
        """
        # import subprocess
        print('Running system command: {0}'.format(' '.join(args_list)))
        proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s_output, s_err = proc.communicate()
        s_return =  proc.returncode
        return s_return, s_output, s_err
ret, paths, err = run_cmd(['hdfs', 'dfs', '-ls', '-C', './CA2/*.csv.gzip'])
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()


Running system command: hdfs dfs -ls -C ./CA2/*.csv.gzip


KeyspaceName = 'ukrainetweets'
TableName = 'RawTweet'
session.set_keyspace(KeyspaceName)
session.execute(f'DROP TABLE {TableName}')

In [3]:
import os
import csv
import gzip
from cassandra.cluster import Cluster
import pandas as pd


ReplicationOptions = {'class': 'SimpleStrategy', 'replication_factor': 1}

query = f"CREATE KEYSPACE IF NOT EXISTS {KeyspaceName} WITH REPLICATION = {ReplicationOptions}"
session.execute(query)

TableName = 'RawTweet'
ColumnNames = ["num", "userid","username","acctdesc","location","following","followers","totaltweets","usercreatedts","tweetid","tweetcreatedts","retweetcount","text","hashtags","language","coordinates","favorite_count","is_retweet","original_tweet_id","original_tweet_userid","original_tweet_username","in_reply_to_status_id","in_reply_to_user_id","in_reply_to_screen_name","is_quote_status","quoted_status_id","quoted_status_userid","quoted_status_username","extractedts"]

create_table = f"CREATE TABLE IF NOT EXISTS {KeyspaceName}.{TableName} ({', '.join([f'{column} TEXT' for column in ColumnNames])}, PRIMARY KEY (tweetid))"
session.execute(create_table)

direct = '/home/hduser/Desktop/CA2/CA2'
I = 0
tweets = []
keywords = ['refugee', 'refugees', 'asylum', 'asylum seeker', 'asylum seeking', 'migration', 'refugee crisis']
for file in os.listdir(direct):
    I+=1
    print(I)
    if file.endswith('.csv.gzip'):
        path = os.path.join(direct, file)
        
        with gzip.open(path, 'rt', encoding = 'utf-8') as file:
            reader = csv.reader(file)
            header = next(reader)
            text_index = header.index('text')
            for row in reader:
                if any(keyword in row[text_index].lower() for keyword in keywords):
                    tweets.append(row)
        
df = pd.DataFrame(tweets)
df.head()

df.to_csv('Filtered_Tweets.csv', index = False)

#The cassandra drivers seem to not support the copy command so this was done in the terminal using the following command
'''
COPY RawTweet (num, userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,is_retweet,original_tweet_id,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts)FROM '/home/hduser/Desktop/CA2/Filtered_Tweets.csv'

'''





1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
count = session.execute('SELECT COUNT(*) FROM RawTweet')
print(count[0])

Row(count=504775)


/tmp/ipykernel_6488/2430243498.py:2: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  print(count[0])


In [10]:
Refugee = session.execute('select tweetid, tweetcreatedts, text from RawTweet ')
Refugee = pd.DataFrame(list(Refugee))
Refugee.head()

,tweetid,tweetcreatedts,text
0,1502751230489153544,2022-03-12 20:59:47,"Safe in Poland, but unwelcome in Britain this ..."
1,1525451387873763328,2022-05-14 12:22:07.000000,.Millions in #Tigray ‘Trying to survive’ as th...
2,1505448856376422404,2022-03-20 07:39:11,A #Ukrainian refugee child comes to a kinderga...
3,1511002990819176456,2022-04-04 15:29:20.000000,#Ukraine #Mariupol #Russia #Belarus #NATO #EU ...
4,1514739676476805130,2022-04-14 22:57:36.000000,#Canada will deploy up to 150 Canadian armed f...


In [17]:
Refugee.shape

(504775, 3)